In [2]:
import numpy as np
import pandas as pd
import random

In [30]:
POPULATION_SIZE = 50
NUM_GENERATIONS = 25
GENES = '''abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '''
TARGET = 'Hello World'
CHROMOSOME_LEN = len(TARGET)
MUTATION_RATE = 0.1

In [31]:
def random_num(start, end):
    range_val = (end - start) + 1
    random_int = start + random.randint(0, range_val - 1)
    return random_int

In [32]:
def mutated_genes():
    length_data = len(GENES)
    random_index = random_num(0, length_data-1)
    return GENES[random_index]

In [33]:
def create_chromosome():
    return ''.join(mutated_genes() for _ in range(CHROMOSOME_LEN))

In [34]:
def cal_fitness(chromosome): 
    fitness = 0
    for gene_char, target_char in zip(chromosome, TARGET):
        if gene_char == target_char:
            fitness += 1
    return fitness 

In [35]:
def create_init_population():
    return [create_chromosome() for _ in range(POPULATION_SIZE)]

In [36]:
def select_parents(population, fitness_scores):
    selected_parents = []
    sorted_indices = np.argsort(fitness_scores)[::-1].tolist()
    selected_population = [population[i] for i in sorted_indices[:POPULATION_SIZE // 2]]
    
    return selected_population

In [37]:
a = create_init_population()
def roulette_wheel_selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    normalized_fitness = [fit / total_fitness for fit in fitness_values]
    wheel = [sum(normalized_fitness[:i+1]) for i in range(len(normalized_fitness))]

    selected_parents = []
    
    for i in range(POPULATION_SIZE // 2):
        selection_point = random.uniform(0, 1)
        selected_index = None
        for i, value in enumerate(wheel):
            if selection_point <= value:
                selected_index = i
                if population[selected_index] not in selected_parents:
                    selected_parents.append(population[selected_index])
                break
                

    return selected_parents

fitness_scores = [cal_fitness(chromosome) for chromosome in a]
print(fitness_scores)
selected_individual = roulette_wheel_selection(a, fitness_scores)
selected_fitness_scores = [cal_fitness(chromosome) for chromosome in selected_individual]
print(selected_fitness_scores)
# print("\nSelected:", selected_individual)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [38]:
def one_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, CHROMOSOME_LEN - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    
    return child1, child2

In [42]:
def two_point_crossover(parent1, parent2):
    crossover_points = sorted(random.sample(range(1, CHROMOSOME_LEN - 1), 2))
    point1, point2 = crossover_points

    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2

In [43]:
a = create_chromosome()
b = create_chromosome()
def k_point_crossover(parent1, parent2):
    K_CROSSOVER_POINTS = 4
    crossover_points = sorted(random.sample(range(1, CHROMOSOME_LEN), K_CROSSOVER_POINTS))
    
    children = []
    current_parent = parent1

    for point in crossover_points:
        children.append(current_parent[:point])
        current_parent = parent2 if current_parent is parent1 else parent1

    children.append(current_parent[point:])
    
    # Ensure child lengths are the same as parent lengths
    child1 = "".join(children[::2]) + parent1[crossover_points[-1]:]
    child2 = "".join(children[1::2]) + parent2[crossover_points[-1]:]
    
    return child1, child2


In [44]:
a = create_chromosome()
b = create_chromosome()
def uniform_crossover(parent1, parent2):
    child1 = [gene1 if random.random() < MUTATION_RATE else gene2 for gene1, gene2 in zip(parent1, parent2)]
    child2 = [gene2 if random.random() < MUTATION_RATE else gene1 for gene1, gene2 in zip(parent1, parent2)]
    
    return "".join(child1), "".join(child2)


In [57]:
a = create_chromosome()
b = create_chromosome()

def pmx_crossover(parent1, parent2):
    crossover_points = sorted(random.sample(range(CHROMOSOME_LEN), 2))
    point1, point2 = crossover_points
    child1 = list(parent1)
    child2 = list(parent2)

    mapping1 = {value: index for index, value in enumerate(parent1)}
    mapping2 = {value: index for index, value in enumerate(parent2)}

    for i in range(CHROMOSOME_LEN):
        if i < point1 or i > point2:
            if parent2[i] not in child1:
                value = parent2[i]
                while value in mapping1:
                    index_in_mapping = mapping1[value]
                    value = parent2[index_in_mapping]
                child1[i] = value
                
            if parent1[i] not in child2:
                value = parent1[i]
                while value in mapping2:
                    index_in_mapping = mapping2[value]
                    value = parent1[index_in_mapping]
                child2[i] = value
                
    positions = list(range(point1, point2+1))   
    for i, position in enumerate(positions):
        child1[position] = parent2[position]
        child2[position] = parent1[position]

    return ''.join(child1), ''.join(child2)

child1, child2 = pmx_crossover(a, b)
print(a, b)
print(child1, child2)

5 8 ['C', 'q', 'k', 'm', 's', 'b', 'I', 'R', 'S', 'h', 'G'] ['y', 'd', 'C', 'Y', ' ', 'e', 'l', 'u', 'C', 'L', 'W'] {'C': 0, 'q': 1, 'k': 2, 'm': 3, 's': 4, 'b': 5, 'I': 6, 'R': 7, 'S': 8, 'h': 9, 'G': 10} {'y': 0, 'd': 1, 'C': 8, 'Y': 3, ' ': 4, 'e': 5, 'l': 6, 'u': 7, 'L': 9, 'W': 10}
CqkmsbIRShG ydCY eluCLW
ydyY eluCLW yqkmsbIRShG


In [46]:
def mutate(chromosome):
    chromosome_list = list(chromosome)
    for i in range(CHROMOSOME_LEN):
        if random.uniform(0, 1) < MUTATION_RATE:
            chromosome_list[i] = mutated_genes()
            
    return ''.join(chromosome_list)

In [47]:
b = create_chromosome()
def displacement_mutation(chromosome):
    chromosome_list = list(chromosome)

    segment_size = random.randint(1, CHROMOSOME_LEN // 2)
    start_position = random.randint(0, CHROMOSOME_LEN - segment_size)
    segment = chromosome_list[start_position:start_position + segment_size]

    del chromosome_list[start_position:start_position + segment_size]
    insert_position = random.randint(0, CHROMOSOME_LEN - segment_size)
    
    result_list = chromosome_list[:insert_position] + segment + chromosome_list[insert_position:]
    return ''.join(result_list)

print(b)
print(displacement_mutation(b))

WmlthYAnpqt
WlthYAnpmqt


In [48]:
b = create_chromosome()
def simple_inversion_mutation(chromosome):

    chromosome_list = list(chromosome)
    position1, position2 = sorted(random.sample(range(CHROMOSOME_LEN), 2))
    inversion_segment = chromosome_list[position1:position2+1]
    inverted_segment = inversion_segment[::-1]
    del chromosome_list[position1:position2+1]

    insert_position = random.randint(0, CHROMOSOME_LEN - len(inverted_segment))
    result_list = chromosome_list[:insert_position] + inverted_segment + chromosome_list[insert_position:]

    return ''.join(result_list)

print(b)
print(simple_inversion_mutation(b))

ssEUVtgsBga
ssEUagBsgtV


In [49]:
b = create_chromosome()
def scramble_mutation(chromosome):
    
    subset_size = random.randint(1, CHROMOSOME_LEN)
    subset_indices = random.sample(range(CHROMOSOME_LEN), subset_size)

    shuffled_subset = random.sample(subset_indices, subset_size)

    mutated_chromosome = list(chromosome)
    for i, index in enumerate(subset_indices):
        mutated_chromosome[index] = chromosome[shuffled_subset[i]]

    return ''.join(mutated_chromosome)

print(b)
print(scramble_mutation(b))

LoTw jrhwfq
LT whqofwjr


In [52]:
def genetic_algorithm():
    population = create_init_population()
    best_solution = population[0]

    while (best_solution != TARGET):
        fitness_scores = [cal_fitness(chromosome) for chromosome in population]
        selected_parents = select_parents(population, fitness_scores)
#         selected_parents = roulette_wheel_selection(population, fitness_scores)

        children = []
        for _ in range(POPULATION_SIZE):
            parent1, parent2 = random.sample(selected_parents, 2)
            child1, child2 = uniform_crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            children.extend([child1, child2])

        population = children
        best_solution = max(population, key=cal_fitness)
        print(best_solution)
    
    return best_solution

In [53]:
optimized_chromosome = genetic_algorithm()
print("Optimized chromosome:", optimized_chromosome)

TePFypTopxp
HStWHOWjfsI
TePWopToFrp
HStWFOWofsX
JePFopWoFSp
HdPFo Wodrv
HeXRo LoQgF
HelVo TopUp
HeXIo WoFgF
HelVo TopTp
HelYo TopUr
HelVo Toplp
Helko Toplp
HelYo ToAll
HelVo WoXXd
HelVo WoZll
HelVo Wo ld
HelVo WoXld
HelVo WoXld
Hello WoXlR
Helco Wonld
Hello WoJld
Hello WoXld
Hello WoUld
Hello WoUld
Hello WoXld
Hello World
Optimized chromosome: Hello World
